# Day 18: Many-Worlds Interpretation

https://adventofcode.com/2019/day/18

## Some notes

This is the last puzzle I'm trying to solve, and possibly the most difficult for me. In the meanwhile, tough, I learned a thing of two about maze searching (e.g. for Day 20) and search algorithms. 

I think the idea of storing the maze in a matrix is not very good, since the keys and doors add an additional dimension to the problem, and the amount of mazes to be stored just t save whther a tile has been visited or note in a BFS search would be of the order of 26! given I have 26 key: too many!

I could represent a point in a search pash as `(x,y,collectedkeys)`, and store the information about the visited cells in a dictionary of a cell. I need to decide how to represent the `collectedkeys` information, it could be a 26-bits word.

## Part 1

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
from queue import Queue

In [2]:
def readInput(inputfile):
    amaze = []
    with open(inputfile) as f:
        amaze = [l.rstrip('\n') for l in f]
    return amaze

Define various bitwise operations to store information about collected keys, and use them to check if a door can be traversed

In [3]:
def setBit(value, bit):
    return value | (1<<bit)

def clearBit(value, bit):
    return value & ~(1<<bit)

def isSet(x, n):
    return x & 1 << n != 0

def keyString(collectedkeys):
    keys = "abcdefghijklmnopqrstuvwxyz"
    keystr = ""
    for i in range(26):
        if isSet(collectedkeys,i):
            keystr += keys[i]
    return keystr

def addKey(collectedkeys,key):
    keys = "abcdefghijklmnopqrstuvwxyz"
    bit = keys.find(key)
    if bit >=0:
        return collectedkeys | (1<<bit)
    else:
        return collectedkeys

def hasKey(key,collectedkeys):
    keys = "abcdefghijklmnopqrstuvwxyz"
    bit = keys.find(key)
    return ( collectedkeys & (1<<bit) ) > 0

def isOpen(door,collectedkeys):
    if door.isupper():
        key = door.lower()
        return hasKey(key,collectedkeys)
    else:
        return False

def allKeys(amaze):
    keys = ""
    for l in amaze:
        for c in l:
            if c.islower():
                keys += c
    keys = ''.join(sorted(keys))
    keyword = 0
    for i in range(len(keys)):
        keyword = keyword | (1<<i)
    return keyword

def getStart(amaze):
    foundStart = False
    ys = 0
    for l in amaze:
        xs = 0
        for c in l:
            if c=="@":
                foundStart = True
                break
            xs += 1
        if foundStart:
            break
        ys += 1
    return (xs,ys,0)

In [4]:
def getAdjacent(n):
    '''returns list of adiacent cells'''
    x,y = n
    return [(x-1,y), # W
            (x,y-1), # N
            (x+1,y), # E
            (x,y+1)] # S

def SearchKeysBFS(start, amaze):
    allkeys = allKeys(amaze)
    queue = Queue()
    queue.put([start])
    visited = [start]
    while not queue.empty():
        path = queue.get()
        c = path[-1]
        xc,yc,collectedkeys = c
        if collectedkeys == allkeys:
            # I don't really need to check all paths for lenght: 
            # BFS will reach the shortest pay with all the keys first
            # so I can simply exit the loop when found!
            print("Found path with all keys collected")
            break
        for a in getAdjacent((xc,yc)):
            xa,ya = a
            ak = (xa,ya,collectedkeys) # adiacent position, current key set
            if amaze[ya][xa] == '#' or ak in visited or \
                ( amaze[ya][xa].isupper() and not isOpen(amaze[ya][xa],collectedkeys) ) : 
                continue
            else:
                # if it's a key I don't already have, collect it
                isKey = amaze[ya][xa].islower() and not hasKey(amaze[ya][xa],collectedkeys)
                new_collectedkeys = collectedkeys
                if isKey:
                    new_collectedkeys = addKey(collectedkeys,amaze[ya][xa])
                ak = (xa,ya,new_collectedkeys)
                visited.append(ak)
                new_path = list(path)                
                new_path.append(ak)
                queue.put(new_path)
                #if isKey:
                    #print(keyString(new_collectedkeys))
    return path

def SearchKeysBFSLenghtOnly(start, amaze):
    allkeys = allKeys(amaze)
    queue = Queue()
    queue.put([0,start]) # saving only path lenght and last visited cell 
    visited = defaultdict(bool) # save visited cell in dictionary for faster access
    visited[start] = True
    while not queue.empty():
        path = queue.get()
        c = path[1]
        xc,yc,collectedkeys = c
        if collectedkeys == allkeys:
            print("Found path with all keys collected")
            break
        for a in getAdjacent((xc,yc)):
            xa,ya = a
            ak = (xa,ya,collectedkeys) # adiacent position, current key set
            if amaze[ya][xa] == '#' or visited[ak] or \
                ( amaze[ya][xa].isupper() and not isOpen(amaze[ya][xa],collectedkeys) ) : 
                continue
            else:
                # if it's a key I don't already have, collect it
                isKey = amaze[ya][xa].islower() and not hasKey(amaze[ya][xa],collectedkeys)
                new_collectedkeys = collectedkeys
                if isKey:
                    new_collectedkeys = addKey(collectedkeys,amaze[ya][xa])
                ak = (xa,ya,new_collectedkeys)
                visited[ak] = True
                lenght = path[0]+1 # incrementing path lenght
                new_path = [lenght,ak] # saving path lenght and last cell position/collectedkeys
                queue.put(new_path)
    return path

In [5]:
## Test 1 -> 8 steps
amaze1 = readInput("./data/day18test1.txt")

path1 = SearchKeysBFS(getStart(amaze1), amaze1)
print("Test 1 - Steps =", len(path1)-1)

path1L = SearchKeysBFSLenghtOnly(getStart(amaze1), amaze1)
print("Test 1 - Steps =", path1L[0])

Found path with all keys collected
Test 1 - Steps = 8
Found path with all keys collected
Test 1 - Steps = 8


In [6]:
## Test 2 -> 86 steps
amaze2 = readInput("./data/day18test2.txt")
path2 = SearchKeysBFS(getStart(amaze2), amaze2)
print("Test 2 - Steps =", len(path2)-1)
path2L = SearchKeysBFSLenghtOnly(getStart(amaze2), amaze2)
print("Test 2 - Steps =", path2L[0])

Found path with all keys collected
Test 2 - Steps = 86
Found path with all keys collected
Test 2 - Steps = 86


In [7]:
## Test 3 -> 132 steps
amaze3 = readInput("./data/day18test3.txt")
path3 = SearchKeysBFS(getStart(amaze3), amaze3)
print("Test 3 - Steps =", len(path3)-1)
path3L = SearchKeysBFSLenghtOnly(getStart(amaze3), amaze3)
print("Test 3 - Steps =", path3L[0])

Found path with all keys collected
Test 3 - Steps = 132
Found path with all keys collected
Test 3 - Steps = 132


In [8]:
## Test 5 -> 81 steps
amaze5 = readInput("./data/day18test5.txt")
path5 = SearchKeysBFS(getStart(amaze5), amaze5)
print("Test 5 - Steps =", len(path5)-1)
path5L = SearchKeysBFSLenghtOnly(getStart(amaze5), amaze5)
print("Test 5 - Steps =", path5L[0])

Found path with all keys collected
Test 5 - Steps = 81
Found path with all keys collected
Test 5 - Steps = 81


In [9]:
# Test 4 -> 136 steps
amaze4 = readInput("./data/day18test4.txt")

# Standard BFS search seems to run forever
#path4 = SearchKeysBFS(getStart(amaze4), amaze4)
#print("Test 4 - Steps =", len(path4)-1)

import time
start_time = time.time()
# Optimized BFS saving only path lenght and last visited cell, 
# and storing visited cells in dictionary for faster access
path4L = SearchKeysBFSLenghtOnly(getStart(amaze4), amaze4)
print("--- %s seconds ---" % (time.time() - start_time))
print("Test 4 - Steps =", path4L[0])

Found path with all keys collected
--- 1.0431249141693115 seconds ---
Test 4 - Steps = 136


In [11]:
amaze = readInput("./data/input18.txt")

start_time = time.time()
pathL = SearchKeysBFSLenghtOnly(getStart(amaze), amaze)
print("--- %s seconds ---" % (time.time() - start_time))
print("Full input - Steps =", pathL[0])

Found path with all keys collected
--- 43.13924789428711 seconds ---
Full input - Steps = 3146
